In [1]:
from ipywidgets import FloatSlider, ValueWidget, Layout
import ipywidgets as widgets

from bqplot import OrdinalScale, LinearScale, Bars, Axis, Figure

import numpy as np

from sidepanel import SidePanel

from regulus.utils import io
from regulus.topo import * 
from regulus.alg import *
from regulus.measures import *
from regulus.models import *
from regulus.tree import *

from ipyregulus import TreeWidget, TreeView, DataWidget
from ipyregulus.sensitivity_view import sensitivity_view
from ipyregulus.fitness_view import fitness_view
from ipyregulus.filters import * 


In [2]:
data = io.load('data/gauss4')

### Initial tree

In [3]:
tw = TreeWidget(data.tree)
p = SidePanel(title='Tree')
tree_view = TreeView(tw, attr='fitness')
tree_view.details = []
with p:
    display(tree_view)

### Test custom widget

In [4]:
sensitivity = sensitivity_view(tree_view)

p2 = SidePanel(title='Sensitivity')
with p2:
    display(sensitivity)

{'new': [0]}


In [5]:
stepwise = fitness_view(tree_view)

p2 = SidePanel(title='Stepwise Fitness')
with p2:
    display(stepwise)

In [6]:
tree_view.tree.regulus.attr['linear'].get(1)[0]

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [7]:
tree_view

TreeView(details=[146, 215, 0], field='fitness', selected=[20, 146, 215], tree_model=TreeWidget(attrs={'fitnes…

In [8]:
import pdir

In [9]:
tree_view.tree.regulus.attr['stepwise_regression']

AttributeError: 'Cache' object has no attribute 'factory'

In [ ]:
model, found = tree_view.tree.regulus.attr['linear'].get(0)

In [ ]:
tree_view.details